In [2]:
### functions
function orthologs(ortholog_table, aliases_table) # iei, it works for everyone ^^ \o/
    dataframe=DataFrame(gene=String[], dmel_homo=String[], string_id=String[])
    for i in 1:length(ortholog_table[1])
        for j in 1:length(aliases_table[2])
            if isna(aliases_table[j,2])
            elseif ortholog_table[i,1] == aliases_table[j,2]
                push!(dataframe, [ortholog_table[i,1], ortholog_table[i,2], aliases_table[j,1]])
            end
        end
    end
    return dataframe
end

function isthere(interactions_table, ortholog1, interactor1) # checks if this pair of interactors is in the table
    for k in 1:length(interactions_table[3])
        if interactions_table[k,3] == ortholog1 && interactions_table[k, 4] == interactor1
            return true
        elseif interactions_table[k,3] == interactor1 && interactions_table[k, 4] == ortholog1
            return true
        else
            continue
        end
    end
    return false
end


function interactions(orthologs_output, interactions_table) # works for all spp \o/ | creates a data frame with all interactions for a given list of genes
    interactions_df=DataFrame(gene=String[], dmel_homo=String[], string_id=String[], interaction_id=String[], interaction_coeficient=Float64[])
    numbers_dic=Dict{String,Int64}()
    for i in 1:length(orthologs_output[3])
        counter = 0
        for j in 1:length(interactions_table[1])
            if orthologs_output[i,3] == interactions_table[j,1]
                if !isthere(interactions_df, orthologs_output[i,3], interactions_table[j,2]) && orthologs_output[i,3]!=interactions_table[j,2]
                    push!(interactions_df, [orthologs_output[i,1], orthologs_output[i,2], orthologs_output[i,3], interactions_table[j,2], interactions_table[j,3]])
                    counter += 1
                end
            elseif orthologs_output[i,3]==interactions_table[j,2]
                if !isthere(interactions_df, orthologs_output[i,3], interactions_table[j,2]) && orthologs_output[i,3]!=interactions_table[j,2]
                    push!(interactions_df, [orthologs_output[i,1], orthologs_output[i,2], orthologs_output[i,3], interactions_table[j,2], interactions_table[j,3]])
                    counter += 1
                end
            end
        end
        numbers_dic[orthologs_output[i,1]]=counter
    end
    return interactions_df, numbers_dic
end

function within(orthologs_otp, interactions_otp)
    numberswithin=Dict{String,Int64}()
    interactionswithin=DataFrame(gene1=String[], dmel_homo1=String[], string_id=String[], interaction_id=String[], gene2=String[], dmel_homo2=String[], interaction_coeficient=Float64[])
    for i in 1:length(orthologs_otp[1])
        counter = 0
        for j in 1:length(interactions_otp[1])
            if interactions_otp[j,1]==orthologs_otp[i,1] && interactions_otp[j,4] in orthologs_otp[3]
                for k in 1:length(orthologs_otp[3])
                    if orthologs_otp[k,3] == interactions_otp[j,4]
                        if !isthere(interactionswithin, interactions_otp[j,3], interactions_otp[j,4])
                            push!(interactionswithin, [interactions_otp[j,1], interactions_otp[j,2], interactions_otp[j,3], interactions_otp[j,4], orthologs_otp[k,1], orthologs_otp[k,2], interactions_otp[j,5]])
                            counter += 1
                        end
                    end
                end
            elseif interactions_otp[j,4]==orthologs_otp[i,3] && interactions_otp[j,3] in orthologs_otp[3]
                counter += 1
            end
        end
        numberswithin[orthologs_otp[i,1]] = counter
    end
    return interactionswithin, numberswithin
end

function proportions1(orthologs_otp, within_int, all_int)
    proportion_otp=Dict{String,Float64}()
    for i in orthologs_otp[1]
        counter = 0
        counter = within_int[i]/all_int[i]
        proportion_otp[i] = counter
    end
    return proportion_otp
end

function numbers_orth(orthologs_otp, interactions_otp)
    numbers_orthologs=Dict{String,Int64}()
    numberswithin_orthologs=Dict{String,Int64}()
    proportions_orth=Dict{String,Float64}()
    for i in 1:length(orthologs_otp[2])
        counter1 = 0
        for j in 1:length(interactions_otp[1])
            if interactions_otp[j,1]==orthologs_otp[i,1]
                counter1 += 1
            end
        end
        if (haskey(numbers_orthologs, orthologs_otp[i,2]))
            numbers_orthologs[orthologs_otp[i,2]] += counter1
        else
            numbers_orthologs[orthologs_otp[i,2]] = counter1
        end
    end
    for i in 1:length(orthologs_otp[2])
        counter2 = 0
        for j in 1:length(interactions_otp[1])
            if interactions_otp[j,1]==orthologs_otp[i,1] && interactions_otp[j,4] in orthologs_otp[3]
                    counter2 += 1
            end
        end
        if !(haskey(numberswithin_orthologs, orthologs_otp[i,2]))
            numberswithin_orthologs[orthologs_otp[i,2]] = counter2
        else
            numberswithin_orthologs[orthologs_otp[i,2]] += counter2
        end
    end
    for i in orthologs_otp[2]
        proportions_orth[i] = numberswithin_orthologs[i]/numbers_orthologs[i]
    end
    return numbers_orthologs, numberswithin_orthologs, proportions_orth
end

numbers_orth (generic function with 1 method)

In [3]:
using DataFrames
celegans = readtable("celegans_experimental.txt", header=false, separator=' ') # open as data frame
c_aliases = readtable("celegans_aliases.txt", header=false, separator='\t', nastrings=["","NA"])
c_orthologs = readtable("celegans_orthologs.txt", header=true, separator='\t')

,_gene,_dmel_homolog
1,rnt-1,runt
2,hlh-14,scute
3,hlh-3,scute
4,eif-3.G,Sex-lethal
5,exc-7,Sex-lethal
6,eya-1,groucho
7,unc-37,groucho
8,wdr-48,groucho
9,wdr-5.3,groucho
10,hlh-2,daughterless


In [4]:
mouse = readtable("mouse_experimental.txt", header=false, separator=' ') # open as data frame
m_aliases = readtable("mouse_aliases.txt", header=false, separator='\t', nastrings=["","NA"])
m_orthologs = readtable("mouse_orthologs.txt", header=true, separator='\t')

,_gene,_dmel_homolog
1,Runx2,runt
2,Runx1,runt
3,Runx3,runt
4,Ascl1,scute
5,Ascl2,scute
6,Ascl3,scute
7,Ascl4,scute
8,Ascl5,scute
9,Ferd3l,scute
10,Figla,scute


In [5]:
yeast = readtable("yeast_experimental.txt", header=false, separator=' ') # open as data frame
y_aliases = readtable("yeast_aliases.txt", header=false, separator='\t', nastrings=["","NA"])
y_orthologs = readtable("yeast_orthologs.txt", header=true, separator='\t')

,_gene,_dmel_homolog
1,GBP2,Sex-lethal
2,PAB1,Sex-lethal
3,TUP1,groucho
4,PHO4,deadpan


In [6]:
dmel = readtable("dmel_experimental.txt", header=false, separator=' ') # open as data frame
d_aliases = readtable("dmel_aliases.txt", header=false, separator='\t', nastrings=["","NA"])
d_orthologs = readtable("dmel_orthologs.txt", header=true, separator='\t')

,_gene_symbol,_dmel_name
1,sisA,sisterless
2,run,runt
3,upd3,unpaired3
4,sc,scute
5,Sxl,Sex-lethal
6,gro,groucho
7,da,daughterless
8,her,hermaphrodite
9,dpn,deadpan
10,tra,transformer


In [7]:
human = readtable("human_experimental.txt", header=false, separator=' ') # open as data frame
h_aliases = readtable("human_aliases.txt", header=false, separator='\t', nastrings=["","NA"])
h_orthologs = readtable("human_orthologs.txt", header=true, separator='\t')

,_gene,_dmel_homolog
1,RUNX1,runt
2,RUNX2,runt
3,RUNX3,runt
4,ASCL1,scute
5,ASCL2,scute
6,ASCL3,scute
7,ASCL4,scute
8,ASCL5,scute
9,FIGLA,scute
10,HAND1,scute


In [8]:
yeast_orthologs = orthologs(y_orthologs, y_aliases)
yeast_interactions, yeast_numbers = interactions(yeast_orthologs, yeast)
yeast_intwithin, yeast_numbwithin = within(yeast_orthologs, yeast_interactions)
yeast_proportion = proportions1(yeast_orthologs, yeast_numbwithin, yeast_numbers)
yeast_numbers_orth, yeast_numbwithin_orth, yeast_proportion_orth = numbers_orth(yeast_orthologs, yeast_interactions)

(Dict("Sex-lethal"=>437,"groucho"=>99,"deadpan"=>66), Dict("Sex-lethal"=>1,"groucho"=>0,"deadpan"=>0), Dict("Sex-lethal"=>0.00228833,"groucho"=>0.0,"deadpan"=>0.0))

In [11]:
length(yeast_interactions[1])

602

In [8]:
celegans_orthologs = orthologs(c_orthologs, c_aliases)
celegans_interactions, celegans_numbers = interactions(celegans_orthologs, celegans)
celegans_intwithin, celegans_numbwithin = within(celegans_orthologs, celegans_interactions)
celegans_proportion = proportions1(celegans_orthologs, celegans_numbwithin, celegans_numbers)
celegans_numbers_orth, celegans_numbwithin_orth, celegans_proportion_orth = numbers_orth(celegans_orthologs, celegans_interactions)

(Dict("Sex-lethal"=>16,"scute"=>19,"groucho"=>154,"runt"=>4,"daughterless"=>15,"deadpan"=>140), Dict("Sex-lethal"=>0,"scute"=>2,"groucho"=>2,"runt"=>0,"daughterless"=>0,"deadpan"=>1), Dict("Sex-lethal"=>0.0,"scute"=>0.105263,"groucho"=>0.012987,"runt"=>0.0,"daughterless"=>0.0,"deadpan"=>0.00714286))

In [9]:
dmel_orthologs = orthologs(d_orthologs, d_aliases)
dmel_interactions, dmel_numbers = interactions(dmel_orthologs, dmel)
dmel_intwithin, dmel_numbwithin = within(dmel_orthologs, dmel_interactions)
dmel_proportion = proportions1(dmel_orthologs, dmel_numbwithin, dmel_numbers)
dmel_numbers_orth, dmel_numbwithin_orth, dmel_proportion_orth = numbers_orth(dmel_orthologs, dmel_interactions)

(Dict("Sex-lethal"=>19,"hermaphrodite"=>1,"scute"=>114,"sisterless"=>7,"unpaired3"=>1,"runt"=>11,"daughterless"=>25,"deadpan"=>6,"groucho"=>38,"transformer"=>5…), Dict("Sex-lethal"=>0,"hermaphrodite"=>0,"scute"=>8,"sisterless"=>3,"unpaired3"=>0,"runt"=>1,"daughterless"=>2,"deadpan"=>0,"groucho"=>1,"transformer"=>0…), Dict("Sex-lethal"=>0.0,"hermaphrodite"=>0.0,"scute"=>0.0701754,"sisterless"=>0.428571,"unpaired3"=>0.0,"runt"=>0.0909091,"daughterless"=>0.08,"deadpan"=>0.0,"groucho"=>0.0263158,"transformer"=>0.0…))

In [10]:
mouse_orthologs = orthologs(m_orthologs, m_aliases)
mouse_interactions, mouse_numbers = interactions(mouse_orthologs, mouse)
mouse_intwithin, mouse_numbwithin = within(mouse_orthologs, mouse_interactions)
mouse_proportion = proportions1(mouse_orthologs, mouse_numbwithin, mouse_numbers)
mouse_numbers_orth, mouse_numbwithin_orth, mouse_proportion_orth = numbers_orth(mouse_orthologs, mouse_interactions)

(Dict("Sex-lethal"=>10,"scute"=>77,"hermaphrodite"=>12,"groucho"=>98,"runt"=>46,"daughterless"=>447,"deadpan"=>31,"emc"=>37), Dict("Sex-lethal"=>1,"scute"=>24,"hermaphrodite"=>0,"groucho"=>4,"runt"=>1,"daughterless"=>33,"deadpan"=>3,"emc"=>1), Dict("Sex-lethal"=>0.1,"scute"=>0.311688,"hermaphrodite"=>0.0,"groucho"=>0.0408163,"runt"=>0.0217391,"daughterless"=>0.0738255,"deadpan"=>0.0967742,"emc"=>0.027027))

In [11]:
human_orthologs = orthologs(h_orthologs, h_aliases)
human_interactions, human_numbers = interactions(human_orthologs, human)
human_intwithin, human_numbwithin = within(human_orthologs, human_interactions)
human_proportion = proportions1(human_orthologs, human_numbwithin, human_numbers)
human_numbers_orth, human_numbwithin_orth, human_proportion_orth = numbers_orth(human_orthologs, human_interactions)

(Dict("Sex-lethal"=>1081,"scute"=>176,"hermaphrodite"=>40,"groucho"=>190,"runt"=>178,"daughterless"=>473,"deadpan"=>213,"emc"=>105), Dict("Sex-lethal"=>9,"scute"=>50,"hermaphrodite"=>0,"groucho"=>9,"runt"=>11,"daughterless"=>27,"deadpan"=>18,"emc"=>1), Dict("Sex-lethal"=>0.00832562,"scute"=>0.284091,"hermaphrodite"=>0.0,"groucho"=>0.0473684,"runt"=>0.0617978,"daughterless"=>0.0570825,"deadpan"=>0.084507,"emc"=>0.00952381))

In [12]:
writetable("yeast_allinteractions1.txt", yeast_interactions, separator='\t', header=true, nastring="NA")
writetable("yeast_queryinteractions1.txt", yeast_intwithin, separator='\t', header=true, nastring="NA")

writetable("celegans_allinteractions1.txt", celegans_interactions, separator='\t', header=true, nastring="NA")
writetable("celegans_queryinteractions1.txt", celegans_intwithin, separator='\t', header=true, nastring="NA")

writetable("dmel_allinteractions1.txt", dmel_interactions, separator='\t', header=true, nastring="NA")
writetable("dmel_queryinteractions1.txt", dmel_intwithin, separator='\t', header=true, nastring="NA")

writetable("mouse_allinteractions1.txt", mouse_interactions, separator='\t', header=true, nastring="NA")
writetable("mouse_queryinteractions1.txt", mouse_intwithin, separator='\t', header=true, nastring="NA")

writetable("human_allinteractions1.txt", human_interactions, separator='\t', header=true, nastring="NA")
writetable("human_queryinteractions1.txt", human_intwithin, separator='\t', header=true, nastring="NA")

In [13]:
yeast_filenumbers1=DataFrame(gene1=String[], dmel_homo=String[], total_interactions=Int64[], withinquery_interactions=Int64[], percentage_withinquery=Float64[])
for i in 1:length(yeast_orthologs)
    push!(yeast_filenumbers1, [yeast_orthologs[i,1],yeast_orthologs[i,2],yeast_numbers[yeast_orthologs[i,1]],yeast_numbwithin[yeast_orthologs[i,1]],yeast_proportion[yeast_orthologs[i,1]]])
end

writetable("yeast_numbers1.txt", yeast_filenumbers1, separator='\t', header=true, nastring="NA")

In [14]:
celegans_filenumbers1=DataFrame(gene1=String[], dmel_homo=String[], total_interactions=Int64[], withinquery_interactions=Int64[], percentage_withinquery=Float64[])
for i in 1:length(celegans_orthologs)
    push!(celegans_filenumbers1, [celegans_orthologs[i,1],celegans_orthologs[i,2],celegans_numbers[celegans_orthologs[i,1]],celegans_numbwithin[celegans_orthologs[i,1]],celegans_proportion[celegans_orthologs[i,1]]])
end

writetable("celegans_numbers1.txt", celegans_filenumbers1, separator='\t', header=true, nastring="NA")

In [15]:
dmel_filenumbers1=DataFrame(gene1=String[], dmel_homo=String[], total_interactions=Int64[], withinquery_interactions=Int64[], percentage_withinquery=Float64[])
for i in 1:length(dmel_orthologs)
    push!(dmel_filenumbers1, [dmel_orthologs[i,1],dmel_orthologs[i,2],dmel_numbers[dmel_orthologs[i,1]],dmel_numbwithin[dmel_orthologs[i,1]],dmel_proportion[dmel_orthologs[i,1]]])
end

writetable("dmel_numbers1.txt", dmel_filenumbers1, separator='\t', header=true, nastring="NA")

In [16]:
mouse_filenumbers1=DataFrame(gene1=String[], dmel_homo=String[], total_interactions=Int64[], withinquery_interactions=Int64[], percentage_withinquery=Float64[])
for i in 1:length(mouse_orthologs)
    push!(mouse_filenumbers1, [mouse_orthologs[i,1],mouse_orthologs[i,2],mouse_numbers[mouse_orthologs[i,1]],mouse_numbwithin[mouse_orthologs[i,1]],mouse_proportion[mouse_orthologs[i,1]]])
end

writetable("mouse_numbers1.txt", mouse_filenumbers1, separator='\t', header=true, nastring="NA")

In [17]:
human_filenumbers1=DataFrame(gene1=String[], dmel_homo=String[], total_interactions=Int64[], withinquery_interactions=Int64[], percentage_withinquery=Float64[])
for i in 1:length(human_orthologs)
    push!(human_filenumbers1, [human_orthologs[i,1],human_orthologs[i,2],human_numbers[human_orthologs[i,1]],human_numbwithin[human_orthologs[i,1]],human_proportion[human_orthologs[i,1]]])
end

writetable("human_numbers1.txt", human_filenumbers1, separator='\t', header=true, nastring="NA")

In [18]:
all_filenumbers=DataFrame(dmel_homo=String[], dmel_total_interactions=Int64[], dmel_withinquery_interactions=Int64[], dmel_percentage_withinquery=Float64[], yeast_total_interactions=Int64[], yeast_withinquery_interactions=Int64[], yeast_percentage_withinquery=Float64[], celegans_total_interactions=Int64[], celegans_withinquery_interactions=Int64[], celegans_percentage_withinquery=Float64[], mouse_total_interactions=Int64[], mouse_withinquery_interactions=Int64[], mouse_percentage_withinquery=Float64[], human_total_interactions=Int64[], human_withinquery_interactions=Int64[], human_percentage_withinquery=Float64[])

for i in 1:length(dmel_orthologs[2])
    if !haskey(yeast_numbers_orth, dmel_orthologs[i,2])
        yeast_numbers_orth[dmel_orthologs[i,2]] = 0
        yeast_numbwithin_orth[dmel_orthologs[i,2]] = 0
        yeast_proportion_orth[dmel_orthologs[i,2]] = 0
    end
    if !haskey(celegans_numbers_orth, dmel_orthologs[i,2])
        celegans_numbers_orth[dmel_orthologs[i,2]] = 0
        celegans_numbwithin_orth[dmel_orthologs[i,2]] = 0
        celegans_proportion_orth[dmel_orthologs[i,2]] = 0
    end
    if !haskey(mouse_numbers_orth, dmel_orthologs[i,2])
        mouse_numbers_orth[dmel_orthologs[i,2]] = 0
        mouse_numbwithin_orth[dmel_orthologs[i,2]] = 0
        mouse_proportion_orth[dmel_orthologs[i,2]] = 0
    end
    if !haskey(human_numbers_orth, dmel_orthologs[i,2])
        human_numbers_orth[dmel_orthologs[i,2]] = 0
        human_numbwithin_orth[dmel_orthologs[i,2]] = 0
        human_proportion_orth[dmel_orthologs[i,2]] = 0
    end
    push!(all_filenumbers, [dmel_orthologs[i,2], dmel_numbers_orth[dmel_orthologs[i,2]], dmel_numbwithin_orth[dmel_orthologs[i,2]], dmel_proportion_orth[dmel_orthologs[i,2]], yeast_numbers_orth[dmel_orthologs[i,2]], yeast_numbwithin_orth[dmel_orthologs[i,2]], yeast_proportion_orth[dmel_orthologs[i,2]], celegans_numbers_orth[dmel_orthologs[i,2]], celegans_numbwithin_orth[dmel_orthologs[i,2]], celegans_proportion_orth[dmel_orthologs[i,2]], mouse_numbers_orth[dmel_orthologs[i,2]], mouse_numbwithin_orth[dmel_orthologs[i,2]], mouse_proportion_orth[dmel_orthologs[i,2]], human_numbers_orth[dmel_orthologs[i,2]], human_numbwithin_orth[dmel_orthologs[i,2]], human_proportion_orth[dmel_orthologs[i,2]]])
end

writetable("allorthologs_numbers1.txt", all_filenumbers, separator='\t', header=true, nastring="NA")